In [136]:
import numpy as np
import matplotlib.pyplot as plt 
from IPython import display

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

import cv2
from os import listdir
from sklearn.utils import shuffle

from numpy.random import seed
seed(1)
tf.random.set_seed(2)

In [2]:
def scale_and_normalize(arr):
    arr = arr.astype('float32')
    mean, stand_dev = arr.mean(), arr.std()
    arr = (arr-mean)/stand_dev
    arr = np.clip(arr, -1, 1)
    arr = (arr+1)/2
    return arr

In [27]:
def load_data(dir_list, image_size):
    """
    Read images, resize and normalize them. 
    Arguments:
        dir_list: list of strings representing file directories.
    Returns:
        X: A numpy array with shape = (#_examples, image_width, image_height, #_channels)
        y: A numpy array with shape = (#_examples, 1)
    """

    # load all images in a directory
    X = []
    y = []
    image_width, image_height = image_size
    
    for directory in dir_list:
        for filename in listdir(directory):
            # load the image
            image = cv2.imread(directory + '/' + filename)
            # crop the brain and ignore the unnecessary rest part of the image
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # resize image
            image = cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
            # normalize values
            image = scale_and_normalize(image)
            # convert image to numpy array and append it to X
            X.append(image)
            # append a value of 1 to the target array if the image
            # is in the folder named 'yes', otherwise append 0.
            if directory[-3:] == 'yes':
                y.append([1])
            else:
                y.append([0])
                
    X = np.array(X)
    y = np.array(y)
    
    # Shuffle the data
    X, y = shuffle(X, y)
    
    print(f'Number of examples is: {len(X)}')
    print(f'X shape is: {X.shape}')
    print(f'y shape is: {y.shape}')
    
    return X, y

In [28]:
IMG_WIDTH, IMG_HEIGHT = (240, 240)
X, y = load_data(['../data/JPG_Brain_Scans/yes', '../data/JPG_Brain_Scans/no'], (IMG_WIDTH, IMG_HEIGHT))

Number of examples is: 253
X shape is: (253, 240, 240)
y shape is: (253, 1)


In [29]:
X = X.reshape(-1, 240, 240, 1)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .33, shuffle=True, stratify=y)

In [31]:
data_augmentation = keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [133]:
data_augmentation

In [150]:
## ended up using this ImageDataGenerator to generate random rotations/flips

datagen = keras.preprocessing.image.ImageDataGenerator(rotation_range=30,
                                                          horizontal_flip=True,
                                                          vertical_flip=True,
                                                          validation_split=.2)

In [155]:
# Apply the generator to the train and test sets

train_generator = datagen.flow(X_train, y_train)

validation_generator = datagen.flow(X_test, y_test)

In [167]:
model = keras.models.Sequential([
    keras.Input(shape=(240, 240, 1)),
    keras.layers.Conv2D(32, 3, strides=(1, 1), activation='relu', data_format='channels_last', name='conv0'),

    #   Added additional convolutional layers after analyzing performance
    keras.layers.Conv2D(64, 3, strides=(1, 1), activation='relu', data_format='channels_last', name='conv1'),
    keras.layers.Conv2D(128, 3, strides=(1, 1), activation='relu', data_format='channels_last', name='conv2'),

    keras.layers.MaxPool2D((2, 2), name='max_pool0'),
    keras.layers.BatchNormalization(name='bn0'),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation='sigmoid')])

In [168]:
model.summary()

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv0 (Conv2D)               (None, 238, 238, 32)      320       
_________________________________________________________________
conv1 (Conv2D)               (None, 236, 236, 64)      18496     
_________________________________________________________________
conv2 (Conv2D)               (None, 234, 234, 128)     73856     
_________________________________________________________________
max_pool0 (MaxPooling2D)     (None, 117, 117, 128)     0         
_________________________________________________________________
bn0 (BatchNormalization)     (None, 117, 117, 128)     512       
_________________________________________________________________
flatten_29 (Flatten)         (None, 1752192)           0         
_________________________________________________________________
dense_29 (Dense)             (None, 1)               

In [169]:
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss = 'binary_crossentropy',
             optimizer = opt, metrics = ['binary_accuracy'])

In [149]:
from datetime import datetime
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_path = 'models/best_classifier.h5'

tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Save best model according to its validation set binary accuracy
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_path,
    monitor='val_binary_accuracy',
    mode='max',
    save_best_only=True)



neural_net = model.fit(X_train, y_train, batch_size=32, epochs = 25, validation_data = (X_test, y_test), callbacks=[tensorboard_callback, model_checkpoint_callback])

Epoch 1/10
6/6 [==============================] - 26s 4s/step - loss: 31.4709 - binary_accuracy: 0.5119 - val_loss: 0.8102 - val_binary_accuracy: 0.6071
Epoch 2/10
6/6 [==============================] - 25s 4s/step - loss: 6.8487 - binary_accuracy: 0.6897 - val_loss: 0.5393 - val_binary_accuracy: 0.7619
Epoch 3/10
6/6 [==============================] - 25s 4s/step - loss: 3.7746 - binary_accuracy: 0.5991 - val_loss: 0.5171 - val_binary_accuracy: 0.7619
Epoch 4/10
6/6 [==============================] - 24s 4s/step - loss: 2.0967 - binary_accuracy: 0.7924 - val_loss: 0.6253 - val_binary_accuracy: 0.6071
Epoch 5/10
6/6 [==============================] - 24s 4s/step - loss: 1.6847 - binary_accuracy: 0.6115 - val_loss: 0.6794 - val_binary_accuracy: 0.6071
Epoch 6/10
6/6 [==============================] - 24s 4s/step - loss: 1.1977 - binary_accuracy: 0.6733 - val_loss: 0.6160 - val_binary_accuracy: 0.7500
Epoch 7/10
6/6 [==============================] - 25s 4s/step - loss: 1.0490 - binary_a

In [170]:
# Test model with rotations/flips, validation accuracy seems significantly worse

neural_net = model.fit_generator(train_generator, steps_per_epoch=6, epochs = 25, 
                                 validation_data=validation_generator, validation_steps=2,
                                 callbacks=[tensorboard_callback])

Epoch 1/25
6/6 [==============================] - 25s 4s/step - loss: 26.4510 - binary_accuracy: 0.5298 - val_loss: 1.5061 - val_binary_accuracy: 0.6094
Epoch 2/25
6/6 [==============================] - 24s 4s/step - loss: 10.9850 - binary_accuracy: 0.7025 - val_loss: 0.5888 - val_binary_accuracy: 0.7500
Epoch 3/25
6/6 [==============================] - 23s 4s/step - loss: 1.9805 - binary_accuracy: 0.7102 - val_loss: 0.6389 - val_binary_accuracy: 0.6562
Epoch 4/25
6/6 [==============================] - 23s 4s/step - loss: 1.8816 - binary_accuracy: 0.6597 - val_loss: 0.6714 - val_binary_accuracy: 0.5469
Epoch 5/25
6/6 [==============================] - 23s 4s/step - loss: 0.7347 - binary_accuracy: 0.7358 - val_loss: 0.6630 - val_binary_accuracy: 0.7344
Epoch 6/25
6/6 [==============================] - 23s 4s/step - loss: 0.4077 - binary_accuracy: 0.8380 - val_loss: 0.7231 - val_binary_accuracy: 0.3906
Epoch 7/25
6/6 [==============================] - 23s 4s/step - loss: 0.4347 - binary_

In [171]:
sum(y_test)/len(y_test)

array([0.60714286])